In [1]:
import cv2
import numpy as np

In [2]:
kalman = cv2.KalmanFilter(4,2)
kalman.measurementMatrix = np.array([[1,0,0,0],
                                    [0,1,0,0]], np.float32)  # 측정 백터의 위치값을 상대 백터로 옮기기 위한 관계 행렬
kalman.transitionMatrix = np.array([[1,0,1,0],             
                                   [0,1,0,1],
                                   [0,0,1,0],                    # 동역학을 모델링하고 상태 변화를 표현하기 위해
                                   [0,0,0,1]], np.float32)       # 현재 상태를 다음 상태 백터로 변환하기 위한 행렬
kalman.processNoiseCov = np.array([[1,0,0,0],
                                  [0,1,0,0],
                                  [0,0,1,0],
                                  [0,0,0,1]], np.float32) * 0.05

In [3]:
cap = cv2.VideoCapture('./data/slow_traffic_small.mp4')

ret, frame = cap.read()
bbox = cv2.selectROI('Select Object', frame, False, False)

# 객체 추적을 위한 초기 추정 위치 설정
kalman.statePre = np.array([[bbox[0]],
                           [bbox[1]],
                           [0],
                           [0]], np.float32)  # array (초기 x 좌표, y좌표, x방향 속도, y방향 속도 [초기값 0])

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    # 칼만 필터를 사용하여 객체 추적 (바운딩 박스의 중심점)
    kalman.correct(np.array([[np.float32(bbox[0] + bbox[2]/2)],
                            [np.float32(bbox[1] + bbox[3]/2)]]))
    kalman.predict()
    
    # 칼만 필터로 추적된 객체 위치
    predicted_bbox = tuple(map(int, kalman.statePost[:2,0]))
    
    # 시각화
    cv2.rectangle(frame, (predicted_bbox[0]-bbox[2] // 2, predicted_bbox[1]- bbox[3] // 2), 
                  (predicted_bbox[0] + bbox[2]//2, predicted_bbox[1] + bbox[3]//2), (0,255,0), 2)
    
    cv2.imshow('Kalman filter tracking', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    